In [75]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# 创建示例数据
ag_df = pd.read_excel('data/ag_df.xlsx')
ag_df = ag_df[ag_df["settle"] != 0]
ag_df.drop("Unnamed: 0", axis=1, inplace=True)
ag_df=ag_df[["date","buy_sell_rate"]]
ag_df

,date,buy_sell_rate
0,2012-05-10,0.260501
1,2012-05-11,-0.899700
2,2012-05-14,-0.368078
3,2012-05-15,0.513435
4,2012-05-16,-1.009386
...,...,...
3005,2024-09-09,-0.788304
3006,2024-09-10,0.666194
3007,2024-09-11,2.034276
3008,2024-09-12,0.684644


In [91]:
# from numba import njit
# @njit
def strategy(df,buy=1.3,sell=-1.5,force_sell = -1.5):

    def cal_op(x):
        if x < force_sell:
            return 'force_sell'
        if x > buy:
            return 'buy'
        elif x < sell:
            return 'sell'
        else:
            return None
    # df['op'] = df['buy_sell_rate'].apply(lambda x: 'buy' if x > buy else ('sell' if x < sell else None))
    # df['op'] = df['buy_sell_rate'].apply(lambda x: 'force_sell' if x < force_sell else x)
    df['op'] = df['buy_sell_rate'].apply(cal_op)
    filtered_df = df[df['op'].notnull()]
    
    # 获取买卖点，并且删除没有买卖点的数据
    buy_df = filtered_df[filtered_df['op'] == 'buy']
    sell_df = filtered_df[filtered_df['op'] != 'buy']
    force_sell_df = filtered_df[filtered_df['op'] == 'force_sell']
    sell_df.reset_index(drop=True, inplace=True)
    buy_df.reset_index(drop=True, inplace=True)
    force_sell_df.reset_index(drop=True, inplace=True)
    # 排序
    sell_dates= sell_df['date']
    sell_dates.sort_values()
    force_sell_dates= force_sell_df['date']
    force_sell_dates.sort_values()
    
    # 计算卖出日期
    def cal_sell_date(buy_date):
        # Filter the DataFrame first
        filtered_dates = sell_df[(force_sell_dates < buy_date + pd.Timedelta(days=30)) & (force_sell_dates > pd.to_datetime(buy_date))]['date'].values

        if filtered_dates.size == 0:
            filtered_dates = sell_df[(sell_dates >= buy_date + pd.Timedelta(days=30)) & (sell_dates <= buy_date + pd.Timedelta(days=50))]['date'].values
        else:
            return filtered_dates[0]
        # Check if the filtered array is not empty
        if filtered_dates.size > 0:
            return filtered_dates[0]
        else:
            return None  # or handle the case where no valid sell date is found
    
    def cal_sell_pr(sell_date):

        filtered_dates = sell_df[sell_df['date']==sell_date]['buy_sell_rate'].values
        
        # Check if the filtered array is not empty
        if filtered_dates.size > 0:
            return filtered_dates[0]
        else:
            return None  # or handle the case where no valid sell date is found
    
    # 为buy_df添加 sell_date 列
    buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
    buy_df['sell_pr'] = buy_df['sell_date'].apply(cal_sell_pr)
    
    return buy_df


filtered_df =strategy(ag_df)
filtered_df

C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\2738915203.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\2738915203.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_pr'] = buy_df['sell_date'].apply(cal_sell_pr)


,date,buy_sell_rate,op,sell_date,sell_pr
0,2013-08-15,3.479576,buy,NaT,NaN
1,2013-08-23,2.560426,buy,NaT,NaN
2,2014-11-17,2.732558,buy,2014-12-01,-2.632394
3,2015-01-21,1.388175,buy,2015-01-30,-1.679261
4,2015-03-12,1.332175,buy,2015-04-09,-1.541096
...,...,...,...,...,...
68,2024-07-31,1.312441,buy,2024-09-02,-1.841340
69,2024-08-12,1.500625,buy,2024-09-02,-1.841340
70,2024-08-15,1.529052,buy,2024-09-02,-1.841340
71,2024-08-26,1.354872,buy,2024-09-02,-1.841340


In [65]:
def calculate_profit(df,buy=1.3,sell=-1):
    filtered_df =strategy(df,buy=buy,sell=sell)
    filtered_df=filtered_df[filtered_df["sell_date"].notna()]
    # na_df=filtered_df[filtered_df["sell_date"].isna()]
    num_rows = len(filtered_df.index)
    profit = num_rows*(buy-sell-1)
    return profit,num_rows,filtered_df

In [62]:
profits =pd.DataFrame()
for buy_pr in np.arange(0.3, 2.5, 0.1):
    for sell_pr in np.arange(-0.3, -2.5, -0.1):
        
        profit,num_rows ,_= calculate_profit(ag_df,buy=buy_pr,sell=sell_pr)
        one = pd.DataFrame([buy_pr, sell_pr, profit,num_rows], index=['buy_pr', 'sell_pr', 'profit',"num_rows"])
        profits = pd.concat([profits, one.T], axis=0)
profits
        

C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\3216956438.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\3216956438.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\3216956438.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,buy_pr,sell_pr,profit,num_rows
0,0.3,-0.3,-2.564000e+02,641.0
0,0.3,-0.4,-1.782000e+02,594.0
0,0.3,-0.5,-1.104000e+02,552.0
0,0.3,-0.6,-5.210000e+01,521.0
0,0.3,-0.7,1.070255e-13,482.0
...,...,...,...,...
0,2.4,-2.0,1.360000e+01,4.0
0,2.4,-2.1,1.400000e+01,4.0
0,2.4,-2.2,1.440000e+01,4.0
0,2.4,-2.3,1.480000e+01,4.0


In [67]:
profit,num_rows,filtered_df = calculate_profit(ag_df,buy=1,sell=-1.2)
filtered_df

C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\3216956438.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)


,date,buy_sell_rate,op,sell_date
4,2013-09-24,1.043084,buy,2013-10-30
8,2014-09-26,1.125065,buy,2014-10-30
11,2014-12-09,1.056539,buy,2015-01-22
12,2014-12-10,1.072018,buy,2015-01-22
13,2014-12-18,1.180196,buy,2015-01-22
...,...,...,...,...
123,2024-04-12,2.408994,buy,2024-05-23
126,2024-05-29,1.346762,buy,2024-07-12
127,2024-06-06,1.406172,buy,2024-07-12
128,2024-06-20,1.272614,buy,2024-07-25


In [43]:
profit

0.0

In [68]:
profits["each_trade_profit"] = profits["profit"]/profits["num_rows"]
profits.to_excel("data/profits.xlsx")